In [1]:
from pynq import Overlay, allocate
from pynq.lib.dma import DMA
import numpy as np
import time

# Load the overlay (bitstream)
overlay = Overlay("spi_dma.bit")

In [2]:
# Access the AXI Quad SPI and AXI DMA IP cores
spi = overlay.axi_quad_spi_0
dma = overlay.axi_dma_0

In [3]:
spi.register_map

Help on DefaultIP in module pynq.overlay object:

class DefaultIP(builtins.object)
 |  Driver for an IP without a more specific driver
 |  
 |  This driver wraps an MMIO device and provides a base class
 |  for more specific drivers written later. It also provides
 |  access to GPIO outputs and interrupts inputs via attributes. More specific
 |  drivers should inherit from `DefaultIP` and include a
 |  `bindto` entry containing all of the IP that the driver
 |  should bind to. Subclasses meeting these requirements will
 |  automatically be registered.
 |  
 |  Attributes
 |  ----------
 |  mmio : pynq.MMIO
 |      Underlying MMIO driver for the device
 |  _interrupts : dict
 |      Subset of the PL.interrupt_pins related to this IP
 |  _gpio : dict
 |      Subset of the PL.gpio_dict related to this IP
 |  
 |  Methods defined here:
 |  
 |  __init__(self, description)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  call(self, *args, **kwargs)
 |  
 |  re

In [4]:
help(dma)

Help on DMA in module pynq.lib.dma object:

class DMA(pynq.overlay.DefaultIP)
 |  Class for Interacting with the AXI Simple DMA Engine
 |  
 |  This class provides two attributes for the read and write channels.
 |  The read channel copies data from the stream into memory and
 |  the write channel copies data from memory to the output stream.
 |  Both channels have an identical API consisting of `transfer` and
 |  `wait` functions. If interrupts have been enabled and connected
 |  for the DMA engine then `wait_async` is also present.
 |  
 |  Buffers to be transferred must be a `PynqBuffer` objectedallocated
 |  through `pynq.allocate()` function either directly or indirectly. This
 |  means that Frames from the video subsystem can be transferred using
 |  this class.
 |  
 |  Attributes
 |  ----------
 |  recvchannel : _DMAChannel
 |      The stream to memory channel
 |  sendchannel : _DMAChannel
 |      The memory to stream channel
 |  
 |  Method resolution order:
 |      DMA
 |    

In [11]:
# SPI register offsets (relative to base address)
SPI_SPICR_OFFSET = 0x60  # Control Register
SPI_SPISR_OFFSET = 0x64  # Status Register
SPI_TXD_OFFSET   = 0x68  # Transmit FIFO
SPI_RXD_OFFSET   = 0x6C  # Receive FIFO
SPI_SPISSR_OFFSET = 0x70  # Slave Select Register

In [12]:
# DMA Buffer size (you can adjust this based on your needs)
buffer_size = 100

In [13]:
# Allocate DMA buffers for transmit and receive
tx_buffer = allocate(shape=(buffer_size,), dtype=np.uint32)
rx_buffer = allocate(shape=(buffer_size,), dtype=np.uint32)

In [14]:
# Initialize transmit buffer with some example data
tx_buffer[:] = np.arange(buffer_size)

In [15]:
# Function to write to SPI registers
def write_spi(offset, value):
    spi.write(offset, value)

In [16]:
# Function to read from SPI registers
def read_spi(offset):
    return spi.read(offset)